# Distilling Knowledge in Multiple Students  using Generative models

In [1]:
# !nvidia-smi

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils as np_utils

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

### Teacher's Training

In [3]:
#Loading and splitting the dataset into train, validation and test
nb_classes = 10

(X_Train, y_Train), (X_test, y_test) = mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)

X_train = X_train.reshape(48000, 28, 28, 1)
X_val = X_val.reshape(12000, 28, 28, 1)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255


#Creating a teacher network
input_shape = (28, 28, 1) # Input shape of each image

teacher = Sequential()
teacher.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
teacher.add(Conv2D(64, (3, 3), activation='relu'))
teacher.add(MaxPooling2D(pool_size=(2, 2)))

teacher.add(Dropout(0.25)) # For reguralization

teacher.add(Flatten())
teacher.add(Dense(256, activation='relu'))
teacher.add(Dense(256, activation='relu', name="dense_1"))

teacher.add(Dropout(0.5)) # For reguralization

teacher.add(Dense(nb_classes, name = 'dense_2'))
teacher.add(Activation('softmax')) # Note that we add a normal softmax layer to begin with

teacher.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

print(teacher.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               2359552   
_________________________________________________________________
dense_1 (Dense)              (None, 256)               6

In [4]:
teacher.load_weights("Teacher_MNIST_98.h5")

In [5]:
# Train the teacher model as usual
epochs = 0
batch_size = 256
teacher.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_val, Y_val))

### Evaluation 

In [6]:
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [7]:
X_test = X_test.reshape(10000, 28, 28, 1)
X_test = X_test.astype('float32')
X_test /= 255

In [8]:
teacher.evaluate(X_test,Y_test)

313/313 [==============================] - 8s 4ms/step - loss: 0.0541 - accuracy: 0.9869


[0.03862640634179115, 0.9898999929428101]

### Dense Vector split

In [9]:
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)

# 8 Students case
# ---------------------------------------------
s1Train=train_dense[:,:32]
s2Train=train_dense[:,32:64]
s3Train=train_dense[:,64:96]
s4Train=train_dense[:,96:128]
s5Train=train_dense[:,128:160]
s6Train=train_dense[:,160:192]
s7Train=train_dense[:,192:224]
s8Train=train_dense[:,224:]

s1Val=val_dense[:,:32]
s2Val=val_dense[:,32:64]
s3Val=val_dense[:,64:96]
s4Val=val_dense[:,96:128]
s5Val=val_dense[:,128:160]
s6Val=val_dense[:,160:192]
s7Val=val_dense[:,192:224]
s8Val=val_dense[:,224:]

### Student

In [10]:
def define_student(name):

    student1 = Sequential()
    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',input_shape=(28, 28, 1),kernel_initializer='normal', name=name))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Conv2D(16, kernel_size=(3, 3),activation='relu',kernel_initializer='normal'))
    student1.add(Conv2D(16, (3, 3), activation='relu',kernel_initializer='normal'))
    student1.add(MaxPooling2D(pool_size=(2, 2)))

    student1.add(Dropout(0.25)) # For reguralization
    student1.add(Flatten())
    student1.add(Dense(12, activation='relu'))
    student1.add(Dropout(0.3))
    student1.add(Dense(32,name='req'+name))

    student1.compile('adam',loss='mean_squared_error',metrics=['accuracy'])
    
    return student1

ss1=define_student("ss1").summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
ss1 (Conv2D)                 (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 16)        2320      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 10, 10, 16)        2320      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 16)          2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 16)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 4, 4, 16)         

In [11]:
s1=define_student("s1")
s2=define_student('s2')
s3=define_student("s3")
s4=define_student('s4')
s5=define_student("s5")
s6=define_student('s6')
s7=define_student("s7")
s8=define_student('s8')


In [12]:
s1.fit([X_train], s1Train, batch_size=128, epochs=20, verbose=1, 
       validation_data=([X_val], s1Val))
s2.fit([X_train], s2Train, batch_size=128, epochs=20, verbose=1, 
       validation_data=([X_val], s2Val))
s3.fit([X_train], s3Train, batch_size=128, epochs=20, verbose=1, 
       validation_data=([X_val], s3Val))
s4.fit([X_train], s4Train, batch_size=128, epochs=20, verbose=1, 
       validation_data=([X_val], s4Val))
s5.fit([X_train], s5Train, batch_size=128, epochs=20, verbose=1, 
       validation_data=([X_val], s5Val))
s6.fit([X_train], s6Train, batch_size=128, epochs=20, verbose=1, 
       validation_data=([X_val], s6Val))
s7.fit([X_train], s7Train, batch_size=128, epochs=20, verbose=1, 
       validation_data=([X_val], s7Val))
s8.fit([X_train], s8Train, batch_size=128, epochs=20, verbose=1, 
       validation_data=([X_val], s8Val))

Epoch 1/20
375/375 [==============================] - 3s 7ms/step - loss: 1.8928 - accuracy: 0.1102 - val_loss: 0.6864 - val_accuracy: 0.5080
Epoch 2/20
375/375 [==============================] - 2s 5ms/step - loss: 0.9235 - accuracy: 0.4090 - val_loss: 0.5006 - val_accuracy: 0.5428
Epoch 3/20
375/375 [==============================] - 3s 8ms/step - loss: 0.7805 - accuracy: 0.4382 - val_loss: 0.4191 - val_accuracy: 0.5871
Epoch 4/20
375/375 [==============================] - 3s 7ms/step - loss: 0.7296 - accuracy: 0.4660 - val_loss: 0.4218 - val_accuracy: 0.5911
Epoch 5/20
375/375 [==============================] - 3s 9ms/step - loss: 0.7026 - accuracy: 0.4709 - val_loss: 0.3958 - val_accuracy: 0.5934
Epoch 6/20
375/375 [==============================] - 3s 9ms/step - loss: 0.6693 - accuracy: 0.4770 - val_loss: 0.3543 - val_accuracy: 0.6054
Epoch 7/20
375/375 [==============================] - 3s 8ms/step - loss: 0.6530 - accuracy: 0.4785 - val_loss: 0.3627 - val_accuracy: 0.6000
Epoch 

375/375 [==============================] - 3s 8ms/step - loss: 0.6661 - accuracy: 0.3968 - val_loss: 0.3818 - val_accuracy: 0.4763
Epoch 19/20
375/375 [==============================] - 4s 10ms/step - loss: 0.6630 - accuracy: 0.3974 - val_loss: 0.3838 - val_accuracy: 0.4952
Epoch 20/20
375/375 [==============================] - 4s 10ms/step - loss: 0.6572 - accuracy: 0.4015 - val_loss: 0.3976 - val_accuracy: 0.4911
Epoch 1/20
375/375 [==============================] - 4s 10ms/step - loss: 1.5652 - accuracy: 0.1210 - val_loss: 0.6995 - val_accuracy: 0.3601
Epoch 2/20
375/375 [==============================] - 4s 10ms/step - loss: 0.8327 - accuracy: 0.3243 - val_loss: 0.4824 - val_accuracy: 0.5155
Epoch 3/20
375/375 [==============================] - 3s 9ms/step - loss: 0.7195 - accuracy: 0.3853 - val_loss: 0.4363 - val_accuracy: 0.5227
Epoch 4/20
375/375 [==============================] - 4s 10ms/step - loss: 0.6618 - accuracy: 0.4089 - val_loss: 0.3782 - val_accuracy: 0.5336
Epoch 5/20

375/375 [==============================] - 4s 10ms/step - loss: 0.5819 - accuracy: 0.3365 - val_loss: 0.3576 - val_accuracy: 0.4466
Epoch 16/20
375/375 [==============================] - 4s 9ms/step - loss: 0.5811 - accuracy: 0.3331 - val_loss: 0.3465 - val_accuracy: 0.4484
Epoch 17/20
375/375 [==============================] - 4s 10ms/step - loss: 0.5771 - accuracy: 0.3345 - val_loss: 0.3523 - val_accuracy: 0.4425
Epoch 18/20
375/375 [==============================] - 4s 10ms/step - loss: 0.5742 - accuracy: 0.3360 - val_loss: 0.3535 - val_accuracy: 0.4468
Epoch 19/20
375/375 [==============================] - 3s 9ms/step - loss: 0.5746 - accuracy: 0.3374 - val_loss: 0.3600 - val_accuracy: 0.4433
Epoch 20/20
375/375 [==============================] - 4s 10ms/step - loss: 0.5724 - accuracy: 0.3374 - val_loss: 0.3401 - val_accuracy: 0.4532
Epoch 1/20
375/375 [==============================] - 4s 9ms/step - loss: 1.5849 - accuracy: 0.0976 - val_loss: 0.7172 - val_accuracy: 0.3252
Epoch 2/

## **8 Students**

In [13]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
o3=s3.get_layer("reqs3").output
o4=s4.get_layer("reqs4").output

o5=s5.get_layer("reqs5").output
o6=s6.get_layer("reqs6").output
o7=s7.get_layer("reqs7").output
o8=s8.get_layer("reqs8").output

output=tensorflow.keras.layers.concatenate([o1,o2,o3,o4, o5,o6,o7,o8])
print (output.shape)
output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)
mm8=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,s3.get_layer("s3").input,s4.get_layer("s4").input,
                  s5.get_layer("s5").input, s6.get_layer("s6").input,s7.get_layer("s7").input,s8.get_layer("s8").input],output3)
my_weights=teacher.get_layer('dense_2').get_weights()

(None, 256)


In [14]:
# i=0
for l in mm8.layers[:len(mm8.layers)-1]:
    l.trainable=False

mm8.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [15]:
batch_size = 128
mm8_history=mm8.fit([X_train,X_train,X_train,X_train,X_train,X_train,X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=10,
          verbose=1, validation_data=([X_val, X_val, X_val, X_val,X_val, X_val, X_val, X_val], Y_val))

Epoch 1/10
375/375 [==============================] - 9s 21ms/step - loss: 0.9941 - accuracy: 0.7135 - val_loss: 0.0487 - val_accuracy: 0.9853
Epoch 2/10
375/375 [==============================] - 8s 20ms/step - loss: 0.1156 - accuracy: 0.9664 - val_loss: 0.0457 - val_accuracy: 0.9851
Epoch 3/10
375/375 [==============================] - 8s 20ms/step - loss: 0.0931 - accuracy: 0.9728 - val_loss: 0.0457 - val_accuracy: 0.9861
Epoch 4/10
375/375 [==============================] - 7s 18ms/step - loss: 0.0833 - accuracy: 0.9747 - val_loss: 0.0447 - val_accuracy: 0.9859
Epoch 5/10
375/375 [==============================] - 7s 19ms/step - loss: 0.0803 - accuracy: 0.9765 - val_loss: 0.0445 - val_accuracy: 0.9859
Epoch 6/10
375/375 [==============================] - 7s 19ms/step - loss: 0.0814 - accuracy: 0.9767 - val_loss: 0.0441 - val_accuracy: 0.9868
Epoch 7/10
375/375 [==============================] - 7s 19ms/step - loss: 0.0711 - accuracy: 0.9798 - val_loss: 0.0441 - val_accuracy: 0.9867

In [16]:
loss, acc = mm8.evaluate([X_test,X_test,X_test,X_test,X_test,X_test,X_test,X_test], Y_test, verbose=1)
loss, acc

313/313 [==============================] - 6s 18ms/step - loss: 0.0346 - accuracy: 0.9899


(0.03456930071115494, 0.9898999929428101)

In [17]:
mm8.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
s1_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s2_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s3_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
s4_input (InputLayer)           [(None, 28, 28, 1)]  0                                            
____________________________________________________________________________________________